1. Read raw data from the Volume

# 📘 Medallion Architecture with Databricks — MovieLens Example  
هذا المثال يوضح تطبيق معمارية **Medallion Architecture** (Bronze → Silver → Gold) باستخدام بيانات MovieLens المخزنة في Volumes على Databricks.

---

## ⭐ مقدمة
تعتمد معمارية Medallion على ثلاث طبقات:
1. **Bronze**: البيانات الخام كما هي (Raw Ingestion).
2. **Silver**: البيانات المنظّفة والمستخرجة (Cleaned & Enriched).
3. **Gold**: بيانات جاهزة للتحليل (Analytics-Ready).

يهدف هذا المثال إلى:
- استيراد بيانات الخام من Volumes  
- إنشاء طبقة Bronze كجداول Managed Delta  
- تنظيف البيانات وربطها في Silver  
- إنتاج جداول Insights جاهزة للاستخدام في Gold  

الملفات المستخدمة متاحة داخل Volume:
`/Volumes/workspace/default/movielens/`

---

# 🥉 Bronze Layer  
طبقة Bronze تحتوي على البيانات الخام دون تعديل، إضافة إلى بعض بيانات الـ metadata مثل وقت الإدخال.

### 1. Bronze – Ratings (`u.data`)
- قراءة ملف التقييمات (4 أعمدة)
- إضافة metadata
- تخزينه في جدول Delta

اسم الجدول:  
`workspace.movielens_bronze.ratings_bronze`

### 2. Bronze – Items (`u.item`)
- قراءة ملف الأفلام (24 عمودًا)
- تنظيف بسيط للأنواع
- إضافة metadata
- تخزينه في جدول Delta

اسم الجدول:  
`workspace.movielens_bronze.items_bronze`

---

# 🥈 Silver Layer  
طبقة Silver تنظّف البيانات، توحّدها، وتحضّرها للانضمام والتحليل.

### 1. Silver – Ratings
- التأكد من أن التقييم بين 1 و 5  
- تحويل الطابع الزمني إلى `timestamp` و `date`  
- إزالة metadata الخاصة بالـ Bronze  

الجدول الناتج:  
`workspace.movielens_silver.ratings_silver`

### 2. Silver – Items
- تحويل تاريخ الإصدار إلى `date`  
- إزالة الأعمدة غير المستخدمة  
- حذف metadata الخاصة بالـ Bronze  

الجدول الناتج:  
`workspace.movielens_silver.items_silver`

### 3. Silver – Unified Movie Ratings
- دمج التقييمات مع معلومات الأفلام  
- إنشاء جدول غني بالسمات:  
  `user_id`, `movie_id`, `movie_title`, `rating`, `rating_date`, `genres`…

الجدول الناتج:  
`workspace.movielens_silver.movie_ratings_silver`

---

# 🥇 Gold Layer  
طبقة Gold تبني جداول Insights جاهزة للعرض في Dashboards أو Power BI أو Databricks SQL.

### 1. Gold – Movie Summary
ملخص شامل لكل فيلم:
- عدد التقييمات  
- متوسط التقييم  
- أعلى وأدنى تاريخ تقييم  
- الانحراف المعياري  

الجدول:  
`workspace.movielens_gold.movie_summary_gold`

### 2. Gold – Popular Movies
الأفلام التي حصلت على 100 تقييم أو أكثر:
- الترتيب حسب متوسط التقييم  
- ثم عدد التقييمات  

الجدول:  
`workspace.movielens_gold.popular_movies_gold`

### 3. Gold – Ratings by Year
اتجاه التقييمات عبر السنوات:
- عدد التقييمات لكل سنة  
- متوسط التقييم لكل سنة  

الجدول:  
`workspace.movielens_gold.ratings_by_year_gold`

---

# 🔍 خلاصة
هذا الـ pipeline يوضح دورة حياة البيانات الكاملة:
Raw → Clean → Analytics  
ويوفّر نموذجًا بسيطًا وقويًا يمكن إعادة استخدامه مع أي مصدر بيانات في Databricks.

يمكن الآن استخدام طبقة Gold لإنشاء:
- Dashboard للعروض التدريبية  
- Power BI أو Tableau Reports  
- Databricks SQL Queries  
- أو حتى تدريب نماذج ML مستقبلًا



In [0]:
from pyspark.sql import types as T
from pyspark.sql import functions as F

# Path inside the Volume shown in your screenshot
raw_u_data_path = "/Volumes/workspace/default/movielens/u.data"

# Schema according to the README: user id | item id | rating | timestamp
u_data_schema = T.StructType([
    T.StructField("user_id", T.IntegerType(), False),
    T.StructField("item_id", T.IntegerType(), False),
    T.StructField("rating", T.IntegerType(), False),
    T.StructField("rating_timestamp", T.LongType(), False),
])

ratings_bronze_df = (
    spark.read
        .option("sep", "\t")
        .schema(u_data_schema)
        .csv(raw_u_data_path)
        .withColumn("ingestion_timestamp", F.current_timestamp())
        .withColumn("source_file", F.lit("u.data"))
        .withColumn("medallion_layer", F.lit("bronze"))
)


2. Create a schema for Bronze tables (once)

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS movielens_bronze")


DataFrame[]

3. Write the Bronze DataFrame as a managed Delta table

In [0]:
bronze_table_name = "workspace.movielens_bronze.ratings_bronze"

ratings_bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(bronze_table_name)


4. Query

In [0]:
%sql
SELECT * FROM movielens_bronze.ratings_bronze LIMIT 10;


user_id,item_id,rating,rating_timestamp,ingestion_timestamp,source_file,medallion_layer
196,242,3,881250949,2025-11-22T13:18:10.294Z,u.data,bronze
186,302,3,891717742,2025-11-22T13:18:10.294Z,u.data,bronze
22,377,1,878887116,2025-11-22T13:18:10.294Z,u.data,bronze
244,51,2,880606923,2025-11-22T13:18:10.294Z,u.data,bronze
166,346,1,886397596,2025-11-22T13:18:10.294Z,u.data,bronze
298,474,4,884182806,2025-11-22T13:18:10.294Z,u.data,bronze
115,265,2,881171488,2025-11-22T13:18:10.294Z,u.data,bronze
253,465,5,891628467,2025-11-22T13:18:10.294Z,u.data,bronze
305,451,3,886324817,2025-11-22T13:18:10.294Z,u.data,bronze
6,86,3,883603013,2025-11-22T13:18:10.294Z,u.data,bronze


u.item: movie id | movie title | release date | video release date | IMDb URL |
unknown | Action | Adventure | Animation | Children | Comedy | Crime |
Documentary | Drama | Fantasy | Film-Noir | Horror | Musical |
Mystery | Romance | Sci-Fi | Thriller | War | Western

Bronze Ingestion for u.item
1. Define Schema

In [0]:
from pyspark.sql import types as T
from pyspark.sql import functions as F

# Path inside your Volume
raw_item_path = "/Volumes/workspace/default/movielens/u.item"

# Schema for u.item (19 columns)
u_item_schema = T.StructType([
    T.StructField("movie_id", T.IntegerType(), False),
    T.StructField("movie_title", T.StringType(), False),
    T.StructField("release_date", T.StringType(), True),
    T.StructField("video_release_date", T.StringType(), True),
    T.StructField("imdb_url", T.StringType(), True),
    T.StructField("unknown", T.IntegerType(), True),
    T.StructField("Action", T.IntegerType(), True),
    T.StructField("Adventure", T.IntegerType(), True),
    T.StructField("Animation", T.IntegerType(), True),
    T.StructField("Children", T.IntegerType(), True),
    T.StructField("Comedy", T.IntegerType(), True),
    T.StructField("Crime", T.IntegerType(), True),
    T.StructField("Documentary", T.IntegerType(), True),
    T.StructField("Drama", T.IntegerType(), True),
    T.StructField("Fantasy", T.IntegerType(), True),
    T.StructField("Film_Noir", T.IntegerType(), True),
    T.StructField("Horror", T.IntegerType(), True),
    T.StructField("Musical", T.IntegerType(), True),
    T.StructField("Mystery", T.IntegerType(), True),
    T.StructField("Romance", T.IntegerType(), True),
    T.StructField("Sci_Fi", T.IntegerType(), True),
    T.StructField("Thriller", T.IntegerType(), True),
    T.StructField("War", T.IntegerType(), True),
    T.StructField("Western", T.IntegerType(), True)
])


2. Read Raw File into DataFrame

In [0]:
item_bronze_df = (
    spark.read
        .option("sep", "|")      # u.item uses pipe | separator
        .schema(u_item_schema)
        .csv(raw_item_path)
        .withColumn("ingestion_timestamp", F.current_timestamp())
        .withColumn("source_file", F.lit("u.item"))
        .withColumn("medallion_layer", F.lit("bronze"))
)


3. Save as Bronze Managed Table

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.movielens_bronze")

item_bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.movielens_bronze.items_bronze")


In [0]:
%sql
SELECT * FROM workspace.movielens_bronze.items_bronze LIMIT 10;


movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,ingestion_timestamp,source_file,medallion_layer
1,Toy Story (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
2,GoldenEye (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
3,Four Rooms (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
4,Get Shorty (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995),0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
5,Copycat (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
6,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),01-Jan-1995,null,http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
7,Twelve Monkeys (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
8,Babe (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
9,Dead Man Walking (1995),01-Jan-1995,null,http://us.imdb.com/M/title-exact?Dead%20Man%20Walking%20(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2025-11-22T13:22:34.654Z,u.item,bronze
10,Richard III (1995),22-Jan-1996,null,http://us.imdb.com/M/title-exact?Richard%20III%20(1995),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,2025-11-22T13:22:34.654Z,u.item,bronze
